In [1]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [2]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/documents.readonly',
          'https://www.googleapis.com/auth/drive.readonly']

In [3]:
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=6666)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

doc_service = build('docs', 'v1', credentials=creds)
drive_service = build('drive', 'v3', credentials=creds)

documents = doc_service.documents()

In [5]:
q = """
mimeType='application/vnd.google-apps.folder'
and name='Covid-19 risk assessments'
"""
folder = drive_service.files().list(q=q).execute()
folder_id = folder['files'][0]['id']

In [30]:
q = '"%s" in parents' % folder_id
children = drive_service.files().list(q=q).execute()

if children.get('nextPageToken') or children.get('incompleteSearch'):
    raise ValueError("I've lost some of the files!")

file_ids = {child['name']: child['id'] for child in children['files']}


In [49]:
files = {k: documents.get(documentId=v).execute() for k, v in file_ids.items()}

In [50]:
def read_paragraph_element(element):
    """Returns the text in the given ParagraphElement.

        Args:
            element: a ParagraphElement from a Google Doc.
    """
    text_run = element.get('textRun')
    if not text_run:
        return ''
    return text_run.get('content')


def read_structural_elements(elements):
    """Recurses through a list of Structural Elements to read a document's text where text may be
        in nested elements.

        Args:
            elements: a list of Structural Elements.
    """
    text = ''
    for value in elements:
        if 'paragraph' in value:
            elements = value.get('paragraph').get('elements')
            for elem in elements:
                text += read_paragraph_element(elem)
        elif 'table' in value:
            # The text in table cells are in nested Structural Elements and tables may be
            # nested.
            table = value.get('table')
            for row in table.get('tableRows'):
                cells = row.get('tableCells')
                for cell in cells:
                    text += read_strucutural_elements(cell.get('content'))
        elif 'tableOfContents' in value:
            # The text in the TOC is also in a Structural Element.
            toc = value.get('tableOfContents')
            text += read_strucutural_elements(toc.get('content'))
    return text

In [53]:
text = {k: read_structural_elements(v['body']['content']) for k, v in files.items()}

In [55]:
text['Drake']

"WIthin bubble, at home\nWithin bubble, outside\nTakeaway (within bubble, at home)\nTakeaway (within bubble, outside in a public park)\nPicnic, own food (without bubble, outside in a private garden)\nPicnic, own food (without bubble, outside in a public park)\nTakeaway (without bubble, outside in a private garden)\nTakeaway (without bubble, outside in a public park)\nUsing loo or walking through at non-bubble home, masked\nUsing public loos\nWalking day (within bubble, with car rental)\nDrinking at a pub (within bubble, outside, masked staff, distanced tables)\nDrinking at a pub (without bubble, outside, masked staff, distanced tables)\nEating at a restaurant (without bubble, outside, masked staff, distanced tables)\nEating at a restaurant (within bubble, inside, masked staff, distanced tables)\nWorking in an office (masked when not at desk, appropriately socially distant, own food)\nLarge (6+) park gathering/picnic, masked, appropriately socially distant, no food sharing, eating away 